In [ ]:
import cv2 as cv
import inaccel.coral as inaccel
import numpy as np
# from IPython.display import Video, HTML
# import math
# from matplotlib import pyplot as plt
import ipywidgets as widgets
# from ipywebrtc import ImageRecorder, VideoStream
import PIL.Image
import io
from tqdm import tqdm
import threading
from queue import Queue
import time

In [ ]:
# Create ndarrays for the static parameters of the request
# To help with scheduling we create one copy of each array for each request
req_num = 4
cameraMA_l = []
cameraMA_r = []
distC_l = []
distC_r = []
irA_l = []
irA_r = []
bm_state_arr = []
for j in range(req_num):
    cameraMA_l.append(inaccel.array([933.173, 0.0, 663.451, 0.0, 933.173, 377.015, 0.0, 0.0, 1.0], dtype=np.float32))
    cameraMA_r.append(inaccel.array([933.467, 0.0, 678.297, 0.0, 933.467, 359.623, 0.0, 0.0, 1.0], dtype=np.float32))

    distC_l.append(inaccel.array([-0.169398, 0.0227329, 0.0, 0.0, 0.0], dtype=np.float32))
    distC_r.append(inaccel.array([-0.170581, 0.0249444, 0.0, 0.0, 0.0], dtype=np.float32))

    irA_l.append(inaccel.array([0.0011976323, -0.0000000019, -0.8153011732, 0.0000000007, 0.0011976994, \
                               -0.4422348617,  0.0000126839,  0.0000001064, 0.9913820905], dtype=np.float32))
    irA_r.append(inaccel.array([0.0011976994,  0.0000000000, -0.8047567905, -0.0000000000, 0.0011976994, \
                               -0.4420566166, -0.0000000000, -0.0000001064,  1.0000392898], dtype=np.float32))

    bm_state_arr.append(inaccel.array([0, 15, 31, 15, 0, 48, 20, 15, 16, 3, 0], dtype=np.int32))


In [ ]:
# The submitter thread reads frames from the video stream, converts them to the sizes that match the kernel,
# creates the requests and submits them to Coral, and then fills the queue
def submitter():
    for i in range(frame_num):
        # try to read a frame
        ret, frame = input_stream.read()
        if(ret==False):
            queue.put((None,None,None,None,None))
            break
            
        # turn the frame into two inaccel ndarrays, one for the left and one for the right image
        frame_left = cv.cvtColor(cv.resize(frame[:,:width,:], (1280, 720)), cv.COLOR_BGR2GRAY)
        frame_right = cv.cvtColor(cv.resize(frame[:,width:,:], (1280, 720)), cv.COLOR_BGR2GRAY)

        # convert the two frames to inaccel ndarrays and allocate the result ndarray
        left_mat = inaccel.array(frame_left)
        right_mat = inaccel.array(frame_right)
        disp_mat = inaccel.ndarray((720, 1280), dtype=np.uint16)

        # create the request
        req = inaccel.request('com.xilinx.vitis.vision.stereoBM')
        req.arg(left_mat)
        req.arg(right_mat)
        req.arg(disp_mat)
        req.arg(cameraMA_l[i%req_num])
        req.arg(cameraMA_r[i%req_num])
        req.arg(distC_l[i%req_num])
        req.arg(distC_r[i%req_num])
        req.arg(irA_l[i%req_num])
        req.arg(irA_r[i%req_num])
        req.arg(bm_state_arr[i%req_num])
        req.arg(np.int32(720))
        req.arg(np.int32(1280))
        
        # try to submit the request
        try:
            session = inaccel.submit(req)
        except RuntimeError:
            print("Request Failed, ensure the Coral manager is running")
            queue.put((None,None,None,None,None))
            break
        
        # check the queue status and if the queue is full wait
        while queue.full():
            time.sleep(0.01)
            
        # put the new session and arrays in the queue
        queue.put((session,left_mat,right_mat,disp_mat,frame))
        
        # update the queue status
        queue_status.value = queue.qsize()/queue_size

In [ ]:

def printer(sync=False):
    for i in tqdm(range(frame_num)):
        start = time.time()*1000
        
        # try to read from the queue
        session,left_mat,right_mat,disp_mat,frame = queue.get()
        if(session==None):
            break
        
        # wait for the session to complete
        inaccel.wait(session)
        
        # normalize the output array
        disp_img = (disp_mat.view(np.ndarray)*(256.0 / 48.0) / (16.0)).astype(np.uint8)
        disp_img = cv.resize(cv.cvtColor(disp_img, cv.COLOR_GRAY2BGR), (height,width))

        # convert the array to an image and write it to the widget
        disp_img = PIL.Image.fromarray(disp_img)
        f = io.BytesIO()
        disp_img.save(f, format='png')
        image_out.value = f.getvalue()
        image_out.width = int(width*4)
        image_out.height = int(height*4)

        # if print_input is true do the same for the input frame
        if(print_input):
            frame_img = PIL.Image.fromarray(frame)
            f = io.BytesIO()
            frame_img.save(f, format='png')
            image_in.value = f.getvalue()
            image_in.width = int(width*4)
            image_in.height = int(height*2)
        
        
        # explicitly delete the inaccel arrays
        # this step is not necessary, as the garbage collector will delete them too
        # but it helps free the corresponding device memory
        del left_mat
        del right_mat
        del disp_mat
        
        # if sync is true, sleep until it's time for the next frame
        if(sync):
            while ((time.time()*1000) - start < millis_per_frame):
                time.sleep(0.000001)
        
        # update queue status
        queue.task_done()

In [ ]:
# settings
input_video = "data/chair021.mp4"
output_video = "data/output.mp4"
queue_size = 16 # not necessary, but it helps manage resources 
sync_output = False # force output framerate to be as close to the same as the input framerate
print_input = False # very computationally expensive

In [ ]:
# open the input file
input_stream = cv.VideoCapture(input_video)
if (input_stream.isOpened()== False): 
      print("Error opening video stream or file")

# get the input's properties
frame_num = int(input_stream.get(cv.CAP_PROP_FRAME_COUNT))
height = int(input_stream.get(cv.CAP_PROP_FRAME_HEIGHT))
width  = int(input_stream.get(cv.CAP_PROP_FRAME_WIDTH)/2)
fps = int(input_stream.get(cv.CAP_PROP_FPS))

print("Video Info:")
print("   Frames: " + str(frame_num))
print("   Dimensions: " + str(height) + "x" + str(width*2))
print("   Framerate: " + str(fps) + " fps")

millis_per_frame = 1000/fps

# initialize the widgets
image_in = widgets.Image()
image_out = widgets.Image()

queue_status = widgets.FloatProgress(description='Queue:',value=0.0, min=0.0, max=1.0)

if print_input:
    images = [image_in,image_out]
else:
    images = [image_out]

image_box = widgets.VBox(images)

# initialize the queue
queue = Queue(maxsize=queue_size)

# initialize the threads
thread1 = threading.Thread(target=printer, args=[sync_output])
thread2 = threading.Thread(target=submitter)

# start displaying the widgets
display(queue_status)
display(image_box)

# start the threads
thread1.start()
thread2.start()

# wait for the threads to complete
queue.join()
thread1.join()
thread2.join()